In [1]:
from pandas import concat,DataFrame
import numpy as np 
import pandas as pd

In [2]:
df = pd.read_csv("city_hour.csv") 
df=df[df['City']=='Chennai']
df.drop(columns=['City','NOx','PM10','SO2','NO','AQI_Bucket','Benzene','Toluene','Xylene'],axis=1,inplace=True)
df.head(5)

,Datetime,PM2.5,NO2,NH3,CO,O3,AQI
187751,2015-01-01 01:00:00,NaN,47.97,6.36,0.92,7.65,NaN
187752,2015-01-01 02:00:00,NaN,12.31,4.93,0.77,10.96,NaN
187753,2015-01-01 03:00:00,NaN,13.72,3.92,1.06,12.14,NaN
187754,2015-01-01 04:00:00,NaN,14.45,4.54,1.54,11.95,NaN
187755,2015-01-01 05:00:00,NaN,20.16,4.85,1.03,11.64,NaN


In [3]:
df.interpolate(limit_direction="both",inplace=True)
df.head(5)

,Datetime,PM2.5,NO2,NH3,CO,O3,AQI
187751,2015-01-01 01:00:00,428.3,47.97,6.36,0.92,7.65,333.0
187752,2015-01-01 02:00:00,428.3,12.31,4.93,0.77,10.96,333.0
187753,2015-01-01 03:00:00,428.3,13.72,3.92,1.06,12.14,333.0
187754,2015-01-01 04:00:00,428.3,14.45,4.54,1.54,11.95,333.0
187755,2015-01-01 05:00:00,428.3,20.16,4.85,1.03,11.64,333.0


In [4]:
df_test=pd.read_csv("Sensor_Data_Full.csv")
df_test.columns=['Datetime','PM2.5','NO2','NH3','CO','O3','AQI']
df_test.head(5)

,Datetime,PM2.5,NO2,NH3,CO,O3,AQI
0,2021-01-17 10:00:00,45.31,NaN,NaN,NaN,3.04,90.0
1,2021-01-17 11:00:00,44.60,NaN,NaN,NaN,7.21,87.0
2,2021-01-17 12:00:00,39.12,NaN,NaN,NaN,14.03,77.0
3,2021-01-17 13:00:00,35.20,NaN,NaN,NaN,20.43,69.0
4,2021-01-17 14:00:00,32.67,NaN,NaN,NaN,21.31,65.0


In [5]:
df_test.interpolate(limit_direction="both",inplace=True)
df_test.head()


,Datetime,PM2.5,NO2,NH3,CO,O3,AQI
0,2021-01-17 10:00:00,45.31,11.52,133.01,1.37,3.04,90.0
1,2021-01-17 11:00:00,44.60,11.52,133.01,1.37,7.21,87.0
2,2021-01-17 12:00:00,39.12,11.52,133.01,1.37,14.03,77.0
3,2021-01-17 13:00:00,35.20,11.52,133.01,1.37,20.43,69.0
4,2021-01-17 14:00:00,32.67,11.52,133.01,1.37,21.31,65.0


In [6]:
aqi_list=df['AQI'].to_list()
aqi_list_test=df_test['AQI'].to_list()


In [7]:
def series_to_supervised(data, n_in, n_out):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('AQI(t-%d)' % (i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(1, n_out):
		cols.append(df.shift(-i))
		names += [('AQI(t+%d)' % (i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	return agg


In [8]:
df1= series_to_supervised(aqi_list, 4, 6)
print(df1)

       AQI(t-4)  AQI(t-3)  AQI(t-2)  AQI(t-1)  AQI(t+1)  AQI(t+2)  AQI(t+3)  \
0           NaN       NaN       NaN       NaN     333.0     333.0     333.0   
1           NaN       NaN       NaN     333.0     333.0     333.0     333.0   
2           NaN       NaN     333.0     333.0     333.0     333.0     333.0   
3           NaN     333.0     333.0     333.0     333.0     333.0     333.0   
4         333.0     333.0     333.0     333.0     333.0     333.0     333.0   
...         ...       ...       ...       ...       ...       ...       ...   
48187     134.0     130.0     120.0     115.0      88.0      89.0      91.0   
48188     130.0     120.0     115.0      90.0      89.0      91.0      92.0   
48189     120.0     115.0      90.0      88.0      91.0      92.0       NaN   
48190     115.0      90.0      88.0      89.0      92.0       NaN       NaN   
48191      90.0      88.0      89.0      91.0       NaN       NaN       NaN   

       AQI(t+4)  AQI(t+5)  
0         333.0     333

In [9]:
df1_test= series_to_supervised(aqi_list_test, 4, 6)
print(df1_test)

      AQI(t-4)  AQI(t-3)  AQI(t-2)  AQI(t-1)  AQI(t+1)  AQI(t+2)  AQI(t+3)  \
0          NaN       NaN       NaN       NaN      87.0      77.0      69.0   
1          NaN       NaN       NaN      90.0      77.0      69.0      65.0   
2          NaN       NaN      90.0      87.0      69.0      65.0      56.0   
3          NaN      90.0      87.0      77.0      65.0      56.0      44.0   
4         90.0      87.0      77.0      69.0      56.0      44.0      36.0   
...        ...       ...       ...       ...       ...       ...       ...   
1689     104.0     106.0     108.0     113.0     111.0     109.0     107.0   
1690     106.0     108.0     113.0     116.0     109.0     107.0     102.0   
1691     108.0     113.0     116.0     111.0     107.0     102.0       NaN   
1692     113.0     116.0     111.0     109.0     102.0       NaN       NaN   
1693     116.0     111.0     109.0     107.0       NaN       NaN       NaN   

      AQI(t+4)  AQI(t+5)  
0         65.0      56.0  
1        

In [10]:
df2 = pd.concat([df.reset_index(drop=True),df1.reset_index(drop=True)], axis=1)
print(df2)

                  Datetime   PM2.5    NO2    NH3    CO     O3    AQI  \
0      2015-01-01 01:00:00  428.30  47.97   6.36  0.92   7.65  333.0   
1      2015-01-01 02:00:00  428.30  12.31   4.93  0.77  10.96  333.0   
2      2015-01-01 03:00:00  428.30  13.72   3.92  1.06  12.14  333.0   
3      2015-01-01 04:00:00  428.30  14.45   4.54  1.54  11.95  333.0   
4      2015-01-01 05:00:00  428.30  20.16   4.85  1.03  11.64  333.0   
...                    ...     ...    ...    ...   ...    ...    ...   
48187  2020-06-30 20:00:00   23.81  11.13  32.97  1.10  42.74   90.0   
48188  2020-06-30 21:00:00   24.84  11.30  33.20  1.11  45.39   88.0   
48189  2020-06-30 22:00:00   23.84  11.05  33.06  0.99  47.36   89.0   
48190  2020-06-30 23:00:00   22.60  11.15  33.45  1.01  50.78   91.0   
48191  2020-07-01 00:00:00   15.95  10.72  33.52  1.02  48.37   92.0   

       AQI(t-4)  AQI(t-3)  AQI(t-2)  AQI(t-1)  AQI(t+1)  AQI(t+2)  AQI(t+3)  \
0           NaN       NaN       NaN       NaN     333.0 

In [11]:
df2_test = pd.concat([df_test.reset_index(drop=True),df1_test.reset_index(drop=True)], axis=1)
print(df2_test)

                 Datetime  PM2.5    NO2     NH3    CO     O3    AQI  AQI(t-4)  \
0     2021-01-17 10:00:00  45.31  11.52  133.01  1.37   3.04   90.0       NaN   
1     2021-01-17 11:00:00  44.60  11.52  133.01  1.37   7.21   87.0       NaN   
2     2021-01-17 12:00:00  39.12  11.52  133.01  1.37  14.03   77.0       NaN   
3     2021-01-17 13:00:00  35.20  11.52  133.01  1.37  20.43   69.0       NaN   
4     2021-01-17 14:00:00  32.67  11.52  133.01  1.37  21.31   65.0      90.0   
...                   ...    ...    ...     ...   ...    ...    ...       ...   
1689  2021-03-29 08:00:00  66.87   7.95   58.00  9.85  28.78  116.0     104.0   
1690  2021-03-29 09:00:00  61.34   6.60   94.02  9.81  25.64  111.0     106.0   
1691  2021-03-29 10:00:00  59.09   7.05   32.10  9.64  23.94  109.0     108.0   
1692  2021-03-29 11:00:00  57.06  11.19  137.40  9.28  19.09  107.0     113.0   
1693  2021-03-29 12:00:00  52.85  16.52  153.18  9.35  20.81  102.0     116.0   

      AQI(t-3)  AQI(t-2)  A

In [12]:
actual_values=[aqi_list_test[-5:]]
print(actual_values)

[[116.0, 111.0, 109.0, 107.0, 102.0]]


In [13]:
dropnan=True
if dropnan:
	df2.dropna(inplace=True)

In [14]:
df2_test['CO'] = df2_test['CO'].fillna(0)
df2_test['NO2'] = df2_test['NO2'].fillna(0)
df2_test['NH3'] = df2_test['NH3'].fillna(0)
dropnan=True
if dropnan:
	df2_test.dropna(inplace=True)

In [15]:
X_train=np.array(df2[['PM2.5','NO2','NH3','CO','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NO2','NH3','CO','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

In [16]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
rows,cols=X_test.shape

from sklearn.ensemble import GradientBoostingRegressor

lin_reg=MultiOutputRegressor(GradientBoostingRegressor(n_estimators=21,max_depth=34)).fit(X_train, y_train)
y_pred=lin_reg.predict(X_test)
pred=[y_pred[rows-1]]
rmse = sqrt(mean_squared_error(pred,actual_values))


In [2]:
print("Forecasted AQI:")
print(pred)

Forecasted AQI:
[array([111.17109378, 116.43804059, 123.26343922, 112.44771263,
       113.89640324])]


In [3]:
print("RMSE:")
print(rmse)

RMSE:
4.5681824168908545


In [31]:
#PM2.5 removed

X_train=np.array(df2[['NO2','NH3','CO','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['NO2','NH3','CO','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])


In [32]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
rows,cols=X_test.shape

from sklearn.ensemble import GradientBoostingRegressor

lin_reg=MultiOutputRegressor(GradientBoostingRegressor(n_estimators=21,max_depth=34)).fit(X_train, y_train)
y_pred=lin_reg.predict(X_test)
pred=[y_pred[rows-1]]
rmse = sqrt(mean_squared_error(pred,actual_values))
print(rmse)


16.586493563802065


In [10]:
#NO2 removed

X_train=np.array(df2[['PM2.5','NH3','CO','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NH3','CO','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])



In [11]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
rows,cols=X_test.shape

from sklearn.ensemble import GradientBoostingRegressor

lin_reg=MultiOutputRegressor(GradientBoostingRegressor(n_estimators=21,max_depth=34)).fit(X_train, y_train)
y_pred=lin_reg.predict(X_test)
pred=[y_pred[rows-1]]
rmse = sqrt(mean_squared_error(pred,actual_values))
print(rmse)


6.452100112445223


In [17]:
#NH3 removed

X_train=np.array(df2[['PM2.5','NO2','CO','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NO2','CO','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])



In [18]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
rows,cols=X_test.shape

from sklearn.ensemble import GradientBoostingRegressor

lin_reg=MultiOutputRegressor(GradientBoostingRegressor(n_estimators=21,max_depth=34)).fit(X_train, y_train)
y_pred=lin_reg.predict(X_test)
pred=[y_pred[rows-1]]
rmse = sqrt(mean_squared_error(pred,actual_values))
print(rmse)


6.3254839778395215


In [24]:
#CO removed

X_train=np.array(df2[['PM2.5','NO2','NH3','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NO2','NH3','O3','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])



In [25]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
rows,cols=X_test.shape

from sklearn.ensemble import GradientBoostingRegressor

lin_reg=MultiOutputRegressor(GradientBoostingRegressor(n_estimators=21,max_depth=34)).fit(X_train, y_train)
y_pred=lin_reg.predict(X_test)
pred=[y_pred[rows-1]]
rmse = sqrt(mean_squared_error(pred,actual_values))
print(rmse)


7.7583073383360475


In [52]:
#O3 removed

X_train=np.array(df2[['PM2.5','NO2','NH3','CO','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NO2','NH3','CO','AQI','AQI(t-4)','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])



In [53]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
rows,cols=X_test.shape

from sklearn.ensemble import GradientBoostingRegressor

lin_reg=MultiOutputRegressor(GradientBoostingRegressor(n_estimators=21,max_depth=34)).fit(X_train, y_train)
y_pred=lin_reg.predict(X_test)
pred=[y_pred[rows-1]]
rmse = sqrt(mean_squared_error(pred,actual_values))
print(rmse)


5.052495691627039


In [62]:
#AQI(t-4) removed

X_train=np.array(df2[['PM2.5','NO2','NH3','CO','O3','AQI','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NO2','NH3','CO','O3','AQI','AQI(t-3)','AQI(t-2)','AQI(t-1)']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])



In [63]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
rows,cols=X_test.shape

from sklearn.ensemble import GradientBoostingRegressor

lin_reg=MultiOutputRegressor(GradientBoostingRegressor(n_estimators=21,max_depth=34)).fit(X_train, y_train)
y_pred=lin_reg.predict(X_test)
pred=[y_pred[rows-1]]
rmse = sqrt(mean_squared_error(pred,actual_values))
print(rmse)


7.795105436311941


In [64]:
#AQI(t-3) removed
X_train=np.array(df2[['PM2.5','NO2','NH3','CO','O3','AQI','AQI(t-2)','AQI(t-1)']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NO2','NH3','CO','O3','AQI','AQI(t-2)','AQI(t-1)']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])



In [65]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
rows,cols=X_test.shape

from sklearn.ensemble import GradientBoostingRegressor

lin_reg=MultiOutputRegressor(GradientBoostingRegressor(n_estimators=21,max_depth=34)).fit(X_train, y_train)
y_pred=lin_reg.predict(X_test)
pred=[y_pred[rows-1]]
rmse = sqrt(mean_squared_error(pred,actual_values))
print(rmse)


11.897223525438129


In [66]:
#AQI(t-2) removed

X_train=np.array(df2[['PM2.5','NO2','NH3','CO','O3','AQI','AQI(t-1)']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NO2','NH3','CO','O3','AQI','AQI(t-1)']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])



In [67]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
rows,cols=X_test.shape

from sklearn.ensemble import GradientBoostingRegressor

lin_reg=MultiOutputRegressor(GradientBoostingRegressor(n_estimators=21,max_depth=34)).fit(X_train, y_train)
y_pred=lin_reg.predict(X_test)
pred=[y_pred[rows-1]]
rmse = sqrt(mean_squared_error(pred,actual_values))
print(rmse)


12.15616352963059


In [70]:
#AQI(t-1) removed

X_train=np.array(df2[['PM2.5','NO2','NH3','CO','O3','AQI']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NO2','NH3','CO','O3','AQI']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])



In [71]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
rows,cols=X_test.shape

from sklearn.ensemble import GradientBoostingRegressor

lin_reg=MultiOutputRegressor(GradientBoostingRegressor(n_estimators=21,max_depth=34)).fit(X_train, y_train)
y_pred=lin_reg.predict(X_test)
pred=[y_pred[rows-1]]
rmse = sqrt(mean_squared_error(pred,actual_values))
print(rmse)


11.647977270761999


In [74]:
#AQI removed

X_train=np.array(df2[['PM2.5','NO2','NH3','CO','O3']])
y_train=np.array(df2[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])

X_test=np.array(df2_test[['PM2.5','NO2','NH3','CO','O3']])
y_test=np.array(df2_test[['AQI(t+1)','AQI(t+2)','AQI(t+3)','AQI(t+4)','AQI(t+5)']])



In [75]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
rows,cols=X_test.shape

from sklearn.ensemble import GradientBoostingRegressor

lin_reg=MultiOutputRegressor(GradientBoostingRegressor(n_estimators=21,max_depth=34)).fit(X_train, y_train)
y_pred=lin_reg.predict(X_test)
pred=[y_pred[rows-1]]
rmse = sqrt(mean_squared_error(pred,actual_values))
print(rmse)


88.10938079568194
